## Generates the poule sheets for Judo+ tournaments  
#### Work by [Gonçalo Sousa](https://github.com/Sousa1909) for [AJDS](https://github.com/AJD-Santarem)  

#### Script 1
---

In [ ]:
import pandas as pd
import os
import yaml
import random

# Names for the output PDFs
tournament_code = "REPLACE_ME"
output_folder = "Output/" + tournament_code

# Create Input Folder if it doesn't exist
if not os.path.exists("Input"):
    os.makedirs("Input")
    print("Input folder does not exist. Generating...")

# Read the Excel file (Masculine and Feminine sheets) without skipping rows

# GROUP 1
draw_group = "1"

df_masc_17_18 = pd.read_excel('Input/REPLACE_ME.xlsx', sheet_name='BEN-17-18-M')
df_fem_17_18 = pd.read_excel('Input/REPLACE_ME.xlsx', sheet_name='BEN-17-18-F')
df_masc_15_16 = pd.read_excel('Input/REPLACE_ME.xlsx', sheet_name='BEN-15-16-M')
df_fem_15_16 = pd.read_excel('Input/REPLACE_ME.xlsx', sheet_name='BEN-15-16-F')

# GROUP 2
#draw_group = "2"

df_masc = pd.read_excel('Input/REPLACE_ME.xlsx', sheet_name='INF-IN-13-14-M')
df_fem = pd.read_excel('Input/REPLACE_ME.xlsx', sheet_name='INF-IN-13-14-F')

Table Manipulation

In [23]:
# Function to process the data (drop the first row, set column names, and sort)
def process_table(df):
    # Set new column names
    df.columns = ['NAME', 'AGE_TIER', 'BIRTH_YEAR', 'WEIGHT', 'CLUB']

    # Add a STATUS column to track grouping
    df['STATUS'] = 'UNGROUPED'

    # Sort by CLUB (ascending) and WEIGHT (ascending)
    df = df.sort_values(by=['AGE_TIER', 'BIRTH_YEAR', 'WEIGHT'], ascending=[True, True, True])

    return df

# Process both tables
if draw_group == "1":
    df_masc_17_18 = process_table(df_masc_17_18)
    df_fem_17_18 = process_table(df_fem_17_18)
    df_masc_15_16 = process_table(df_masc_15_16)
    df_fem_15_16 = process_table(df_fem_15_16)
else:
    df_masc = process_table(df_masc)
    df_fem = process_table(df_fem)

Athlete Grouping and Draw

In [ ]:
# Helper function to check if a group has an athlete from the same club
def has_same_club(group, club):
    return any(athlete['CLUB'] == club for athlete in group)

# Group athletes by AGE_TIER and weight, with outlier detection
def group_by_age_and_weight(df, weight_range=4.0):
    grouped_by_age_tier = []
    all_outliers = []

    for age_tier, group in df.groupby('AGE_TIER'):
        print(f"Grouping athletes in {age_tier}...")
        
        groups_by_weight, outliers = group_by_weight_and_divide(df, group, weight_range)
        grouped_by_age_tier.append((age_tier, groups_by_weight))
        all_outliers.extend(outliers)

    return grouped_by_age_tier, all_outliers

# Function to group athletes by 4kg weight ranges and detect outliers
def group_by_weight_and_divide(df, df_subset, weight_range):
    column_order = column_order = ['NAME', 'AGE_TIER', 'BIRTH_YEAR', 'WEIGHT', 'CLUB', 'STATUS']
    df_subset['WEIGHT'] = pd.to_numeric(df_subset['WEIGHT'], errors='coerce')

    df_subset = df_subset.sort_values(by='WEIGHT').reset_index()
    
    groups = []
    outliers = []
    current_group = []
    current_limit = df_subset.iloc[0]['WEIGHT'] + weight_range
    
    for _, row in df_subset.iterrows():
        # Check if the current athlete's weight is within the allowed range
        if row['WEIGHT'] <= current_limit:
            current_group.append(row)
        else:
            # Check if the current group has valid members to form a subgroup
            if len(current_group) >= 2:
                groups.append(current_group)
                df.loc[[athlete['index'] for athlete in current_group], 'STATUS'] = 'GROUPED'
            else:
                # If only one athlete is left in a group, consider them as a potential outlier
                outliers.extend(current_group)
                df.loc[[athlete['index'] for athlete in current_group], 'STATUS'] = 'OUTLIER'
            
            # Start a new group with the current athlete
            current_group = [row]
            current_limit = row['WEIGHT'] + weight_range
    
    # Handle the last group
    if len(current_group) >= 2:
        groups.append(current_group)
        df.loc[[athlete['index'] for athlete in current_group], 'STATUS'] = 'GROUPED'
    else:
        outliers.extend(current_group)
        for athlete in current_group:
            df.loc[[athlete['index'] for athlete in current_group], 'STATUS'] = 'OUTLIER'

    # Divide each weight group into subgroups
    target_group_sizes = [5, 4, 3, 2]
    #target_group_sizes = [10, 9, 8, 7, 6, 5, 4, 3, 2]
    final_groups = divide_into_groups(df, groups, column_order, target_group_sizes)

    return final_groups, outliers

# Function to divide athletes into ideal group sizes
def divide_into_groups(df, groups, column_order, target_group_sizes):
    final_groups = []

    for group in groups:
        random.shuffle(group)
        i = 0

        while i < len(group):
            remaining = len(group) - i
            
            for size in target_group_sizes:
                if remaining >= size or remaining == 1:
                    subgroup = group[i:i + size]
                    # Special handling for single-athlete groups
                    if len(subgroup) == 1:
                        # Mark single athletes as OUTLIERS
                        df.loc[[athlete['index'] for athlete in subgroup], 'STATUS'] = 'OUTLIER'
                    else:
                        # Regular group processing
                        df.loc[[athlete['index'] for athlete in subgroup], 'STATUS'] = 'GROUPED'
                        final_groups.append(pd.DataFrame(subgroup)[column_order])
                    i += size
                    break
            else:
                # Remaining athletes are ungrouped
                ungrouped = group[i:]
                if len(ungrouped) == 1:  # Handle single-athlete groups explicitly
                    df.loc[[athlete['index'] for athlete in ungrouped], 'STATUS'] = 'OUTLIER'
                else:
                    df.loc[[athlete['index'] for athlete in ungrouped], 'STATUS'] = 'UNGROUPED'
                break
        
    return final_groups

def handle_unallocated(df):
    unallocated = df[df['STATUS'] == 'UNGROUPED']
    if not unallocated.empty:
        return unallocated
    # Return an empty DataFrame instead of None
    return pd.DataFrame(columns=df.columns)

# Save grouped athletes to YAML with outliers
def save_grouped_athletes_to_yaml(groups_by_age, outliers, ungrouped, gender):
    os.makedirs(output_folder, exist_ok=True)
    output_yaml_file = f'{output_folder}/.grouped_athletes_{gender}.yaml'

    grouped_data = {}

    for age_tier, groups in groups_by_age:
        if "-" in age_tier and age_tier.split("-")[0].isdigit():
            age_tier = f"Benjamim {age_tier}"
        grouped_data[age_tier] = []  # Initialize each age tier section
        for i, group in enumerate(groups):
            group_dict = {
                f'Group {i + 1}': [
                    {
                        'Name': athlete['NAME'],
                        'Weight': athlete['WEIGHT'],
                        'Club': athlete['CLUB'],
                        'Birth Year': athlete['BIRTH_YEAR'],
                    }
                    for _, athlete in group.iterrows()
                ]
            }
            grouped_data[age_tier].append(group_dict)

    # Add outliers to the YAML structure with complete information
    if len(outliers) or len(ungrouped) >= 1:
        save_outliers_and_ungrouped_to_yaml(outliers, ungrouped, gender)

    # Save the structured data as YAML
    with open(output_yaml_file, 'w', encoding='utf-8') as f:
        yaml.dump(grouped_data, f, allow_unicode=True, sort_keys=False)

    print(f"Grouped athletes for {gender} have been saved to {output_yaml_file}.")


def save_outliers_and_ungrouped_to_yaml(outliers, ungrouped, gender):
    output_yaml_file = f'{output_folder}/.outliers_and_ungrouped_{gender}.yaml'
    
    # Structure to hold outliers and ungrouped data for YAML
    data = {
        'Outliers': [
            {
                'Name': athlete['NAME'],
                'Club': athlete['CLUB'],
                'Age Tier': athlete['AGE_TIER'],
                'Birth Year': athlete['BIRTH_YEAR'],
                'Weight': athlete['WEIGHT']
            }
            for athlete in outliers
        ],
        'Ungrouped': [
            {
                'Name': athlete['NAME'],
                'Club': athlete['CLUB'],
                'Age Tier': athlete['AGE_TIER'],
                'Birth Year': athlete['BIRTH_YEAR'],
                'Weight': athlete['WEIGHT']
            }
            for athlete in ungrouped
        ] if ungrouped else []  # Only include Ungrouped if it's not empty
    }

    # Save outliers data as YAML
    with open(output_yaml_file, 'w', encoding='utf-8') as f:
        yaml.dump(data, f, allow_unicode=True, sort_keys=False)

    print(f"Outliers and ungrouped athletes for {gender} have been saved to {output_yaml_file}.")

# Process both tables and include outliers in the output
if draw_group == "1":
    # 2015 - 2016
    groups_fem_15_16, outliers_fem_15_16 = group_by_age_and_weight(df_fem_15_16)
    ungrouped_fem_15_16 = handle_unallocated(df_fem_15_16).to_dict('records')
    save_grouped_athletes_to_yaml(groups_fem_15_16, outliers_fem_15_16, ungrouped_fem_15_16, '15_16_fem')

    groups_masc_15_16, outliers_masc_15_16 = group_by_age_and_weight(df_masc_15_16)
    ungrouped_masc_15_16 = handle_unallocated(df_masc_15_16).to_dict('records')
    save_grouped_athletes_to_yaml(groups_masc_15_16, outliers_masc_15_16, ungrouped_masc_15_16, '15_16_masc')

    # 2017 - 2018
    groups_fem_17_18, outliers_fem_17_18 = group_by_age_and_weight(df_fem_17_18)
    ungrouped_fem_17_18 = handle_unallocated(df_fem_17_18).to_dict('records')
    save_grouped_athletes_to_yaml(groups_fem_17_18, outliers_fem_17_18, ungrouped_fem_17_18, '17_18_fem')

    groups_masc_17_18, outliers_masc_17_18 = group_by_age_and_weight(df_masc_17_18)
    ungrouped_masc_17_18 = handle_unallocated(df_masc_17_18).to_dict('records')
    save_grouped_athletes_to_yaml(groups_masc_17_18, outliers_masc_17_18, ungrouped_masc_17_18, '17_18_masc')

else:
    groups_fem, outliers_fem = group_by_age_and_weight(df_fem)
    ungrouped_fem = handle_unallocated(df_fem).to_dict('records')
    save_grouped_athletes_to_yaml(groups_fem, outliers_fem, ungrouped_fem, 'IN_fem')

    groups_mas, outliers_mas = group_by_age_and_weight(df_masc)
    ungrouped_masc = handle_unallocated(df_masc).to_dict('records')
    save_grouped_athletes_to_yaml(groups_mas, outliers_mas, ungrouped_masc, 'IN_masc')

#### Save tables to HTML

In [ ]:
def df_to_html(df, genre_tag):

    html_file = os.path.join(output_folder, f".final_table_{genre_tag}.html")
    df = df.sort_values(by=['STATUS', 'AGE_TIER', 'BIRTH_YEAR'], ascending=[False, True, True])
    df.to_html(html_file)
    print(f".final_table_{genre_tag} saved to {html_file}")
    
if draw_group == "1":
    df_to_html(df_fem_15_16, "15_16_fem")
    df_to_html(df_fem_17_18, "17_18_fem")
    
    df_to_html(df_masc_15_16, "15_16_mas")
    df_to_html(df_masc_17_18, "17_18_mas")
else:
    df_to_html(df_fem, "IN_fem")
    df_to_html(df_masc, "IN_mas")

#### Generate Summary

In [ ]:
def generate_summary_report(df, groups, outliers, ungrouped, gender):
    total_athletes = len(df)
    grouped_count = sum(len(group) for _, group_list in groups for group in group_list)
    outliers_count = len(outliers)
    ungrouped_count = len(ungrouped) if ungrouped is not None else 0

    print(f"\nSummary Report for {gender.capitalize()}:")
    print(f"-----------------------------------")
    print(f"Total Athletes (Initial): {total_athletes}")
    print(f"Grouped Athletes:         {grouped_count}")
    print(f"Outliers:                 {outliers_count}")
    print(f"Ungrouped:                {ungrouped_count}")
    print(f"-----------------------------------")
    print(f"Check: {grouped_count + outliers_count + ungrouped_count == total_athletes}")


if draw_group == "1":
    generate_summary_report(df_fem_15_16, groups_fem_15_16, outliers_fem_15_16, ungrouped_fem_15_16, '15_16_fem')
    generate_summary_report(df_fem_17_18, groups_fem_17_18, outliers_fem_17_18, ungrouped_fem_17_18, '17_18_fem')

    generate_summary_report(df_fem_15_16, groups_fem_15_16, outliers_fem_15_16, ungrouped_fem_15_16, '15_16_mas')
    generate_summary_report(df_fem_17_18, groups_fem_17_18, outliers_fem_17_18, ungrouped_fem_17_18, '17_18_mas')

else:
    generate_summary_report(df_fem, groups_fem, outliers_fem, ungrouped_fem, 'IN_fem')
    generate_summary_report(df_masc, groups_mas, outliers_mas, ungrouped_masc, 'IN_mas')